# Autogen ಸಹಿತ Agentic RAG

ಈ ನೋಟು Autogen ಏಜೆಂಟ್‌ಗಳನ್ನು ಬಳಸಿಕೊಂಡು Retrieval-Augmented Generation (RAG) ಅನ್ನು ಸುಧಾರಿತ ಮೌಲ್ಯಮಾಪನ ಸಾಮರ್ಥ್ಯಗಳೊಂದಿಗೆ ಅನುಷ್ಠಾನಗೊಳಿಸುವುದನ್ನು ತೋರಿಸುತ್ತದೆ.


SQLite ಆವೃತ್ತಿ ದೋಷ ಸರಿಪಡನೆ  
ನೀವು ಈ ದೋಷವನ್ನು ಎದುರಿಸಿದರೆ:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```
  
ನಿಮ್ಮ ನೋಟುಬುಕ್‌ನ ಪ್ರಾರಂಭದಲ್ಲಿ ಈ ಕೋಡ್ ಬ್ಲಾಕ್ ಅನ್ನು ಅನ್ಕಾಮೆಂಟ್ ಮಾಡಿ:  


In [2]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
!pip install chromadb

In [3]:
import os
import time
import asyncio
from typing import List, Dict
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient

import chromadb

load_dotenv()

True

## ಕ್ಲೈಂಟ್ ಅನ್ನು ರಚಿಸಿ

ಮೊದಲು, ನಾವು Azure AI ಚಾಟ್ ಕಂಪ್ಲೀಷನ್ ಕ್ಲೈಂಟ್ ಅನ್ನು ಪ್ರಾರಂಭಿಸುತ್ತೇವೆ. ಈ ಕ್ಲೈಂಟ್ ಅನ್ನು ಬಳಕೆದಾರರ ಪ್ರಶ್ನೆಗಳಿಗೆ ಪ್ರತಿಕ್ರಿಯೆಗಳನ್ನು ರಚಿಸಲು Azure OpenAI ಸೇವೆಯೊಂದಿಗೆ ಸಂವಹನ ಮಾಡಲು ಬಳಸಲಾಗುತ್ತದೆ.


In [4]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

## ವೆಕ್ಟರ್ ಡೇಟಾಬೇಸ್ ಪ್ರಾರಂಭ

ನಾವು ChromaDB ಅನ್ನು ಸ್ಥಿರ ಸಂಗ್ರಹಣೆಯೊಂದಿಗೆ ಪ್ರಾರಂಭಿಸುತ್ತೇವೆ ಮತ್ತು ಉತ್ತಮ ಮಾದರಿ ಡಾಕ್ಯುಮೆಂಟ್‌ಗಳನ್ನು ಸೇರಿಸುತ್ತೇವೆ. ChromaDB ಅನ್ನು ಡಾಕ್ಯುಮೆಂಟ್‌ಗಳನ್ನು ಸಂಗ್ರಹಿಸಲು ಮತ್ತು ಪುನಃಪ್ರಾಪ್ತಿಗೊಳಿಸಲು ಬಳಸಲಾಗುತ್ತದೆ, ಇದು ನಿಖರವಾದ ಪ್ರತಿಸ್ಪಂದನೆಗಳನ್ನು ರಚಿಸಲು ಸಂದರ್ಭವನ್ನು ಒದಗಿಸುತ್ತದೆ.


In [5]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# Enhanced sample documents
documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours."
]

# Add documents with metadata
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

## ಕಾಂಟೆಕ್ಸ್ಟ್ ಪ್ರೊವೈಡರ್ ಅನುಷ್ಠಾನ

`ContextProvider` ಕ್ಲಾಸ್ ಹಲವು ಮೂಲಗಳಿಂದ ಕಾಂಟೆಕ್ಸ್ಟ್ ಹಿಂಪಡೆಯುವ ಮತ್ತು ಒಕ್ಕೂಟಗೊಳಿಸುವ ಕಾರ್ಯವನ್ನು ನಿರ್ವಹಿಸುತ್ತದೆ:

1. **ವೆಕ್ಟರ್ ಡೇಟಾಬೇಸ್ ಹಿಂಪಡೆಯುವುದು**: ChromaDB ಅನ್ನು ಬಳಸಿಕೊಂಡು ಪ್ರವಾಸದ ಡಾಕ್ಯುಮೆಂಟ್‌ಗಳ ಮೇಲೆ ಅರ್ಥಪೂರ್ಣ ಶೋಧನೆ ನಡೆಸುತ್ತದೆ
2. **ಹವಾಮಾನ ಮಾಹಿತಿ**: ಪ್ರಮುಖ ನಗರಗಳಿಗಾಗಿ ನಕಲಿ ಹವಾಮಾನ ಡೇಟಾಬೇಸ್ ಅನ್ನು ನಿರ್ವಹಿಸುತ್ತದೆ
3. **ಒಕ್ಕೂಟಗೊಳಿಸಿದ ಕಾಂಟೆಕ್ಸ್ಟ್**: ಡಾಕ್ಯುಮೆಂಟ್ ಮತ್ತು ಹವಾಮಾನ ಡೇಟಾವನ್ನು ಸಮಗ್ರ ಕಾಂಟೆಕ್ಸ್ಟ್‌ಗೆ ಒಕ್ಕೂಟಗೊಳಿಸುತ್ತದೆ

ಮುಖ್ಯ ವಿಧಾನಗಳು:
- `get_retrieval_context()`: ಕ್ವೆರಿ ಆಧಾರಿತವಾಗಿ ಸಂಬಂಧಿತ ಡಾಕ್ಯುಮೆಂಟ್‌ಗಳನ್ನು ಹಿಂಪಡೆಯುತ್ತದೆ
- `get_weather_data()`: ನಿರ್ದಿಷ್ಟ ಸ್ಥಳಗಳಿಗಾಗಿ ಹವಾಮಾನ ಮಾಹಿತಿಯನ್ನು ಒದಗಿಸುತ್ತದೆ
- `get_unified_context()`: ಡಾಕ್ಯುಮೆಂಟ್ ಮತ್ತು ಹವಾಮಾನ ಕಾಂಟೆಕ್ಸ್ಟ್‌ಗಳನ್ನು ಒಕ್ಕೂಟಗೊಳಿಸಿ ಉತ್ತಮ ಪ್ರತಿಸ್ಪಂದನೆಗಳನ್ನು ಒದಗಿಸುತ್ತದೆ


In [6]:
class ContextProvider:
    def __init__(self, collection):
        self.collection = collection
        # Simulated weather database
        self.weather_database = {
            "new york": {"temperature": 72, "condition": "Partly Cloudy", "humidity": 65, "wind": "10 mph"},
            "london": {"temperature": 60, "condition": "Rainy", "humidity": 80, "wind": "15 mph"},
            "tokyo": {"temperature": 75, "condition": "Sunny", "humidity": 50, "wind": "5 mph"},
            "sydney": {"temperature": 80, "condition": "Clear", "humidity": 45, "wind": "12 mph"},
            "paris": {"temperature": 68, "condition": "Cloudy", "humidity": 70, "wind": "8 mph"},
        }
    
    def get_retrieval_context(self, query: str) -> str:
        """Retrieves relevant documents from vector database based on query."""
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_strings = []
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_strings.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_strings) if context_strings else "No relevant documents found"
    
    def get_weather_data(self, location: str) -> str:
        """Simulates retrieving weather data for a given location."""
        if not location:
            return ""
            
        location_key = location.lower()
        if location_key in self.weather_database:
            data = self.weather_database[location_key]
            return f"Weather for {location.title()}:\n" \
                   f"Temperature: {data['temperature']}°F\n" \
                   f"Condition: {data['condition']}\n" \
                   f"Humidity: {data['humidity']}%\n" \
                   f"Wind: {data['wind']}"
        else:
            return f"No weather data available for {location}."
    
    def get_unified_context(self, query: str, location: str = None) -> str:
        """Returns a unified context combining both document retrieval and weather data."""
        retrieval_context = self.get_retrieval_context(query)
        
        weather_context = ""
        if location:
            weather_context = self.get_weather_data(location)
            weather_intro = f"\nWeather Information for {location}:\n"
        else:
            weather_intro = ""
        
        return f"Retrieved Context:\n{retrieval_context}\n\n{weather_intro}{weather_context}"

## ಏಜೆಂಟ್ ಸಂರಚನೆ

ನಾವು ರಿಟ್ರೀವಲ್ ಮತ್ತು ಸಹಾಯಕ ಏಜೆಂಟ್‌ಗಳನ್ನು ಸಂರಚಿಸುತ್ತೇವೆ. ರಿಟ್ರೀವಲ್ ಏಜೆಂಟ್ ಸೆಮ್ಯಾಂಟಿಕ್ ಹುಡುಕಾಟವನ್ನು ಬಳಸಿಕೊಂಡು ಸಂಬಂಧಿತ ಮಾಹಿತಿಯನ್ನು ಹುಡುಕುವಲ್ಲಿ ಪರಿಣತಿಯನ್ನು ಹೊಂದಿದೆ, ಆದರೆ ಸಹಾಯಕವು ರಿಟ್ರೀವ್ ಮಾಡಿದ ಮಾಹಿತಿಯ ಆಧಾರದ ಮೇಲೆ ವಿವರವಾದ ಉತ್ತರಗಳನ್ನು ರಚಿಸುತ್ತದೆ.


In [7]:
# Create agents with enhanced capabilities
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message=(
        "You are a helpful AI assistant that provides answers using ONLY the provided context. "
        "Do NOT include any external information. Base your answer entirely on the context given below."
    ),
)

## RAG ಬಳಸಿ ಪ್ರಶ್ನೆ ಪ್ರಕ್ರಿಯೆ

ನಾವು `ask_rag` ಫಂಕ್ಷನ್ ಅನ್ನು ಸಹಾಯಕರಿಗೆ ಪ್ರಶ್ನೆಯನ್ನು ಕಳುಹಿಸಲು, ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ಪ್ರಕ್ರಿಯೆಗೊಳಿಸಲು ಮತ್ತು ಅದನ್ನು ಮೌಲ್ಯಮಾಪನ ಮಾಡಲು ವ್ಯಾಖ್ಯಾನಿಸುತ್ತೇವೆ. ಈ ಫಂಕ್ಷನ್ ಸಹಾಯಕರೊಂದಿಗೆ ಸಂವಹನವನ್ನು ನಿರ್ವಹಿಸುತ್ತದೆ ಮತ್ತು ಪ್ರತಿಕ್ರಿಯೆಯ ಗುಣಮಟ್ಟವನ್ನು ಅಳೆಯಲು ಮೌಲ್ಯಮಾಪಕವನ್ನು ಬಳಸುತ್ತದೆ.


In [8]:
async def ask_rag_agent(query: str, context_provider: ContextProvider, location: str = None):
    """
    Sends a query to the assistant agent with context from the provider.
    
    Args:
        query: The user's question
        context_provider: The context provider instance
        location: Optional location for weather queries
    """
    try:
        # Get unified context
        context = context_provider.get_unified_context(query, location)
        
        # Augment the query with context
        augmented_query = (
            f"{context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide a helpful answer."
        )

        # Send the augmented query to the assistant
        start_time = time.time()
        response = await assistant.on_messages(
            [TextMessage(content=augmented_query, source="user")],
            cancellation_token=CancellationToken(),
        )
        processing_time = time.time() - start_time
        
        return {
            'query': query,
            'response': response.chat_message.content,
            'processing_time': processing_time,
            'location': location
        }
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

# ಉದಾಹರಣೆಯ ಬಳಕೆ

ನಾವು ಮೌಲ್ಯಮಾಪಕವನ್ನು ಪ್ರಾರಂಭಿಸಿ, ಪ್ರಕ್ರಿಯೆಗೊಳಿಸಲು ಮತ್ತು ಮೌಲ್ಯಮಾಪನೆ ಮಾಡಲು ಬಯಸುವ ಪ್ರಶ್ನೆಗಳನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುತ್ತೇವೆ.


In [ ]:
async def main():
    # Initialize context provider
    context_provider = ContextProvider(collection)
    
    # Example queries
    queries = [
        {"query": "What does Contoso's travel insurance cover?"},
        {"query": "What's the weather like in London?", "location": "london"},
        {"query": "What luxury destinations does Contoso offer and what's the weather in Paris?", "location": "paris"},
    ]
    
    print("=== Autogen RAG Demo ===")
    for query_data in queries:
        query = query_data["query"]
        location = query_data.get("location")
        
        print(f"\n\nQuery: {query}")
        if location:
            print(f"Location: {location}")
        
        # Show the context being used
        context = context_provider.get_unified_context(query, location)
        print("\n--- Context Used ---")
        print(context)
        print("-------------------")
        
        # Get response from the agent
        result = await ask_rag_agent(query, context_provider, location)
        if result:
            print(f"\nResponse: {result['response']}")
        print("\n" + "="*50)

## ಸ್ಕ್ರಿಪ್ಟ್ ಅನ್ನು ಚಲಾಯಿಸಿ

ನಾವು ಸ್ಕ್ರಿಪ್ಟ್ ಇಂಟರಾಕ್ಟಿವ್ ಪರಿಸರದಲ್ಲಿ ಅಥವಾ ಸಾಮಾನ್ಯ ಸ್ಕ್ರಿಪ್ಟ್‌ನಲ್ಲಿ ಚಲಿಸುತ್ತಿದೆಯೇ ಎಂದು ಪರಿಶೀಲಿಸುತ್ತೇವೆ ಮತ್ತು ಮುಖ್ಯ ಫಂಕ್ಷನ್ ಅನ್ನು ತದನುಗುಣವಾಗಿ ಚಲಾಯಿಸುತ್ತೇವೆ.


In [12]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        await main()
    else:
        asyncio.run(main())

=== Autogen RAG Demo ===


Query: What does Contoso's travel insurance cover?

--- Context Used ---
Retrieved Context:
Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


-------------------

Response: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.



Query: What's the weather like in London?
Location: london

--- Context Used ---
Retrieved Context:
Document: Popular destinations include the Maldives, Swiss Alps, and African safaris.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


Weather Information for london:
Weather for London:
T

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸಮೀಕ್ಷೆ**:  
ಈ ದಾಖಲೆ AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ಖಚಿತತೆಯನ್ನು ಸಾಧಿಸಲು ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ದಯವಿಟ್ಟು ಗಮನಿಸಿ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸಡ್ಡೆಗಳು ಇರಬಹುದು. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಾಖಲೆ ಪ್ರಾಮಾಣಿಕ ಮೂಲವಾಗಿ ಪರಿಗಣಿಸಬೇಕು. ಪ್ರಮುಖ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪುಅರ್ಥಗಳು ಅಥವಾ ತಪ್ಪುಅರ್ಥೈಸುವಿಕೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
